In [1]:
from dotenv import load_dotenv
from huggingface_hub import login
import os

load_dotenv()
hftoken = os.getenv("HUGGINGFACE_TOKEN")
login(hftoken)

In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
from litellm import completion, batch_completion
import os
import litellm
import time
from tqdm import tqdm

# OpenAI API key 선언
#os.environ["OPENAI_API_KEY"] = "sk-xxx..."

1. query 컬럼으로 질문 생성 및 생성된 질문에 대한 답변 생성

In [8]:

def random_wait(min_seconds=0.1, max_seconds=0.5):
    wait_time = random.uniform(min_seconds, max_seconds)
    #print(f"Waiting for {wait_time:.2f} seconds...")
    time.sleep(wait_time)

# 데이터셋 로드
ds = load_dataset('amphora/rewrite-se-quant')['train']

# 전체 데이터셋을 미리 데이터프레임으로 변환
df_ds = pd.DataFrame(ds)

# 데이터프레임 초기화
df = pd.DataFrame(columns=['query', 'question', 'response'])

batch_size = 1
total_length = len(ds['query'])
csv_file = 'output_dataset.csv'

# 반복문 안에서 현재 배치의 데이터프레임 생성 및 기존 데이터프레임에 추가 8300 ~ 8800까지 다시하기
for i in tqdm(range(15630, total_length, batch_size), desc="Processing Batches", unit="batch"):
    batch_end = min(i + batch_size, total_length)  # 끝 인덱스가 전체 길이를 넘지 않도록 설정

    # 질문 생성용 prompt 포맷팅
    batch_qrys = []
    for t in df_ds['query'][i:batch_end]:
        messages = [
            {"content": "Your job is creating quantitative finance questions in fluent Korean. You will be given a English QF question collected from the web. Restructure it to a test-like question, in formal Korean language. Return the question only.", "role": "system"},
            {"content": t, "role": "user"}
        ]
        batch_qrys.append(messages)

    print(i, 'question creating...')

    # 질문 생성
    responses = batch_completion(
        model="gpt-4o-mini-2024-07-18",
        messages=batch_qrys,
        # max_tokens=60
    )

    question_resps = []
    for response in responses:
        if isinstance(response, litellm.RateLimitError):
            print(f"오류 발생: {str(response)}")
            question_resps.append("Error: Rate limit exceeded or other issue")
        elif hasattr(response, 'choices') and len(response.choices) > 0:
            question_resps.append(response.choices[0].message.content)
        else:
            question_resps.append("Error: Unexpected response format")

    # 질문 생성과 답변 생성 사이에 10초 간격 추가
    # print(i, '1분 대기')
    # time.sleep(60)
    random_wait()

    print(i, 'response creating...')
    # 답변 생성용 prompt 포맷팅
    batch_qrys = []
    for t in question_resps:
        messages = [
            {"content": "You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself.", "role": "system"},
            {"content": t, "role": "user"}
        ]
        batch_qrys.append(messages)

    # 답변 생성
    responses = batch_completion(
        model="gpt-4o-mini-2024-07-18",
        messages=batch_qrys,
        # max_tokens=80
    )

    response_resps = []
    for resp in responses:  # 'i' 대신 'resp'로 변수명 변경
        if isinstance(resp, Exception):
            print(f"오류 발생: {str(resp)}")
            response_resps.append("Error: Rate limit exceeded or other issue")
        elif hasattr(resp, 'choices') and len(resp.choices) > 0:
            response_resps.append(resp.choices[0].message.content)
        else:
            response_resps.append("Error: Unexpected response format")


    # print(i, '1분 대기')
    # time.sleep(60)
    # print('대기 종료')
    random_wait()

    query_list = df_ds['query'].iloc[i:batch_end].tolist()

    # 현재 배치의 데이터프레임 생성 및 기존 데이터프레임에 추가
    batch_df = pd.DataFrame({
        'query': query_list,
        'question': question_resps,
        'response': response_resps
    })

    #batch_df.to_csv(csv_file, mode='a', index=False, header=False, encoding='utf-8-sig')
    
    df = pd.concat([df, batch_df], ignore_index=True)

    # 다음 배치를 처리하기 전 1분 간격 대기
    # print(i, '1초 대기')
    # time.sleep(1)
    # print('대기 종료')
    random_wait()

# 최종 결과 확인
df.head()

Processing Batches:   0%|          | 0/6320 [00:00<?, ?batch/s]

15630 question creating...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

오류 발생: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
15630 response creating...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

오류 발생: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/do

Processing Batches:   0%|          | 1/6320 [00:06<10:41:28,  6.09s/batch]

15631 question creating...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

오류 발생: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
15631 response creating...


Processing Batches:   0%|          | 1/6320 [00:09<16:05:45,  9.17s/batch]


KeyboardInterrupt: 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



In [4]:
df

,query,question,response


In [7]:
dff[:9000].to_csv(csv_file, mode='w', index=False, header=False, encoding='utf-8-sig')

In [9]:
csv_file = 'output_dataset.csv'
len(pd.read_csv(csv_file))

8211

In [8]:
dff = pd.read_csv(csv_file)
dff

,"Since I am very new to this topic, i am struggling to connect the option pricing with Spot Trading. \nIs there a way to use the Black-Scholes model to derive entry and exit Events? Any blogs or papers?\nI've just started to research the Black-Scholes model of using online courses on coursera and somehow fail to find the reasoning for this approach besides option price calculation.\nThanks!!","당신이 옵션 가격 책정과 현물 거래의 연관성을 이해하는 데 어려움을 겪고 있다고 하셨습니다. 블랙-숄즈 모델을 활용하여 진입 및 청산 신호를 도출할 수 있는 방법이 있는지, 혹은 이에 대한 논문이나 블로그 자료가 있는지 질문하십시오. 또한 블랙-숄즈 모델에 대한 연구를 시작했으며, 옵션 가격 계산 외에 이 접근 방식의 배경에 대한 설명을 찾지 못했다고 언급하십시오.","블랙-숄즈 모델을 활용하여 진입 및 청산 신호를 도출하는 방법은 여러 가지가 있을 수 있으며, 이와 관련된 연구나 자료도 존재합니다. 이 모델은 옵션 가격을 계산하는 데 주로 사용되지만, 이를 기반으로 한 트레이딩 전략이나 신호 생성에 대한 연구도 진행되고 있습니다.\n\n우선, 블랙-숄즈 모델의 기본 원리를 이해한 후, 그 원리를 활용하여 시장의 변동성과 상대적 가격 수준을 분석하는 방법을 고려할 수 있습니다. 예를 들어, 옵션의 델타, 감마와 같은 그리스 문자를 분석하여 트레이딩 신호를 유도할 수 있습니다. 또한 옵션의 가격과 기초자산의 가격 간의 상관관계를 분석함으로써 특정 패턴을 포착하는 데 도움을 받을 수 있습니다.\n\n추가적으로 이러한 접근 방식을 다룬 연구 논문이나 블로그 자료는 금융 및 옵션 거래 관련 웹사이트, 금융 전문 잡지, 그리고 학술 데이터베이스에서 찾아볼 수 있습니다. 다양한 트레이딩 전략에 대한 사례와 함께 블랙-숄즈 모델을 적용한 내용이 포함된 자료들을 검색해보는 것이 좋습니다.\n\n더불어, 블랙-숄즈 모델 이외에도 옵션 거래에 대한 추가적인 배경 지식과 다른 가격 책정 모델들에 대한 연구도 활용해보시면 도움이 될 것입니다. 이러한 자료들은 향후 시장 접근 방식을 더욱 넓혀줄 것입니다."
0,If I was to do a 99% VaR calculation on a port...,"포트폴리오의 수익률이 정규 분포 $\mathcal{N} (\mu, \sigma)$를...",1. **정의 및 99% VaR의 설명**\n VaR(가치-at-위험)은 주어진...
1,I'm writing my thesis on the relationship betw...,재무 성과와 지속 가능성 간의 관계에 관한 논문을 작성하고 있습니다. 이를 위해 두...,ESG 변수를 효과적으로 구축하기 위해 다음과 같은 방법을 고려해 보세요.\n\n1...
2,What sources of financial and economic data ar...,온라인에서 이용 가능한 금융 및 경제 데이터의 출처는 무엇인가요? 그 중 무료 또는...,"온라인에서 이용 가능한 금융 및 경제 데이터의 출처는 다양하며, 그 중 일부는 무료..."
3,Assume for now we are working in a stohastic v...,현재 우리는 확률적 변동성 모델(SV 설정)에서 작업하고 있다고 가정합시다. 다음과...,당신의 질문은 통계적 변동성 모델에서 옵션 가격의 민감도를 분석하는 데 중요한 포인...
4,Let $X_t:=e^{W_t}$ where $W_t$ follows the Wie...,확률 과정 $W_t$를 따르는 Wiener 과정에 대해 $X_t := e^{W_t}...,"제시된 풀이 과정은 이토의 보조정리와 연쇄법칙을 적절히 적용하였으며, 드리프트를 구..."
...,...,...,...
8994,I'm working on a project where we're trying to...,"CMS(상수 만기 스왑)와 OIS(하룻밤 지수 스왑)가 서로 다른 개념인지, 혹은 ...",CMS(상수 만기 스왑)와 OIS(하룻밤 지수 스왑)는 본질적으로 서로 다른 금융 ...
8995,I see that the Fama-French library offers annu...,"Fama-French 모델의 연간 요인 데이터가 제공되는데도 불구하고, 왜 많은 사...",Fama-French 모델의 연간 요인 데이터가 제공되더라도 많은 연구자들이 월간 ...
8996,If there is deflation does the Inflation recei...,"디플레이션이 발생할 경우, 인플레이션 수신자는 고정금리 지급 외에도 감소된 소비자물...","디플레이션이 발생하면, 소비자물가지수(CPI)가 감소하게 되고, 이는 전반적인 가격..."
8997,I am aware that I can create a IRS in Quantlib...,주어진 함수 `makeSwap`을 이용하여 이자율 스왑(IRS)을 생성할 수 있음을...,기준 금리 스왑(basis swap)을 생성하기 위해서는 `makeSwap` 함수 ...


In [12]:
dff[dff['response'] == 'Error: Rate limit exceeded or other issue']

,query,question,response
780,Question: All Black-Scholes assumptions hold. ...,"문제: 모든 블랙-숄즈 가정이 성립하며, 배당금은 없다고 가정합니다. 현재 주가는 ...",Error: Rate limit exceeded or other issue
782,"In the cash bond and future basis trade, the n...",현금 채권과 선물 기초 거래에서 순 기초는 옵션(품질 옵션 및 시간 옵션) 프리미엄...,Error: Rate limit exceeded or other issue
783,A major theme in the markets this past week ha...,지난 주 시장에서 주요한 주제는 레포 금리 인상과 유동성의 갑작스러운 감소였습니다....,Error: Rate limit exceeded or other issue
784,"In the process of asking this question, I acut...","주어진 질문에서, 옵션 pricing을 위한 가정된 금리가 시장에서 암시되는 금리와...",Error: Rate limit exceeded or other issue
785,I want to calculate the Turnover of my scaled ...,아래와 같이 질문을 재구성하였습니다:\n\nBarroso와 Santa-Clara (...,Error: Rate limit exceeded or other issue
786,"Suppose I have a strategy, I run a backtest on...",주어진 전략이 있습니다. 이 전략에 대해 하나의 심볼(가정: 백테스트에 사용된 역사...,Error: Rate limit exceeded or other issue
788,Has anyone thought of the relationship between...,옵션의 변동성과 옵션 지급 분포 간의 관계에 대해 생각해본 적이 있습니까? 예를 들...,Error: Rate limit exceeded or other issue
790,I was reading an article and i saw this :\n\nF...,유로존 외부에 있는 자산운용사들은 유럽의 마이너스 수익률 정부 채권을 매입함으로써 ...,Error: Rate limit exceeded or other issue
791,Currently I am reading Basic Black Scholes: Op...,"현재 ""Basic Black Scholes: Option Pricing and Tr...",Error: Rate limit exceeded or other issue
792,Currently I am reading Basic Black Scholes: Op...,"다음 내용의 의미를 파악하기 위한 질문입니다.\n\n티모시 팔콘 크랙의 ""기본 블랙...",Error: Rate limit exceeded or other issue


In [8]:
dff = dff[dff['response'] != 'Error: Rate limit exceeded or other issue']

dff

,query,question,response
0,"This is somewhat non-technical question, but i...",선물 및 옵션과 같은 파생상품이 주식이나 채권보다 더 많은 대체 자산을 갖는 이유는...,파생상품인 선물 및 옵션이 주식이나 채권보다 더 많은 대체 자산을 갖는 이유는 여러...
1,"Since I am very new to this topic, i am strugg...",당신이 옵션 가격 책정과 현물 거래의 연관성을 이해하는 데 어려움을 겪고 있다고 하...,블랙-숄즈 모델을 활용하여 진입 및 청산 신호를 도출하는 방법은 여러 가지가 있을 ...
2,If I was to do a 99% VaR calculation on a port...,"포트폴리오의 수익률이 정규 분포 $\mathcal{N} (\mu, \sigma)$를...",1. **정의 및 99% VaR의 설명**\n VaR(가치-at-위험)은 주어진...
3,I'm writing my thesis on the relationship betw...,재무 성과와 지속 가능성 간의 관계에 관한 논문을 작성하고 있습니다. 이를 위해 두...,ESG 변수를 효과적으로 구축하기 위해 다음과 같은 방법을 고려해 보세요.\n\n1...
4,What sources of financial and economic data ar...,온라인에서 이용 가능한 금융 및 경제 데이터의 출처는 무엇인가요? 그 중 무료 또는...,"온라인에서 이용 가능한 금융 및 경제 데이터의 출처는 다양하며, 그 중 일부는 무료..."
...,...,...,...
16747,I've identified 4 stocks that I think will gai...,다음 질문에 답하십시오:\n\n특정 분기 보고서 발표 시 4개의 주식이 각각 10%...,"제한된 자본으로 수익을 극대화하는 방법에는 몇 가지 전략이 있습니다. 우선, 각 주..."
16748,I was wondering why someone would go to the tr...,어떤 상황에서 경로 의존성이 없는 경우에도 무작위 변수를 생성하는 이유에 대해 설명...,경로 의존성이 없는 경우에도 무작위 변수를 생성하는 이유는 주로 복잡한 금융 자산의...
16749,"I have a list of possible transforms, and I've...",다음은 데이터 전처리 및 변환 작업의 순서에 대한 질문입니다. 주어진 작업 목록에 ...,데이터 전처리 및 변환 작업의 적합한 순서는 다음과 같이 제안할 수 있습니다:\n\...
16750,I am analyzing the nth moment generation proce...,다음은 CIR 모델에 의해 정의된 동적 과정 $r_t$의 n번째 모멘트 생성 과정에...,"CIR 모델의 문맥에서, 함수 \( f_{\lambda}(t, r) \)는 주어진 ..."


In [48]:
df_no_duplicates = pd.concat([df_no_duplicates, df], ignore_index=True)

df_no_duplicates

,query,question,response
0,Vanilla options are traded interbank with delt...,"바닐라 옵션이 인터뱅크에서 델타 헤지를 통해 거래될 때, 델타 헤지를 위해 일반적으...","바닐라 옵션 거래에서 델타 헤지란, 옵션 포지션의 델타를 중립화하여 가격 변동에 의..."
1,"Can the Hodges-Tompkins adjustment (S. Hodges,...","Hodges-Tompkins 조정(S. Hodges, R. Tompkins, The...",Hodges-Tompkins 조정은 중복 관측치로부터 계산된 추정량의 편향을 제거할...
2,To price a call/put option with two possible f...,"다음 질문에 답하십시오:\n\n1) 콜/풋 옵션의 가격을 결정할 때, 왜 항상 기초...",1) 콜/풋 옵션의 가격을 결정할 때 기초 자산과 무위험 채권으로 구성된 복제 포트...
3,I am trying to understand the abbreviations in...,블룸버그 유럽 주식 또는 STOXX 데이터에서 사용하는 약어에 대해 이해하려고 합니...,유럽의 다양한 증권 거래소에서 사용되는 두 글자 약어에 대한 정보를 찾고 계시군요....
4,Assume a model defined in a incomplete market....,불완전한 시장에서 정의된 모델을 가정하십시오. 이 모델에는 몇 가지 매개변수 $\t...,시장 위험 프리미엄 $\lambda$가 관찰할 수 없는 주관적인 매개변수라는 것은 ...
...,...,...,...
413,So I was trying to estimate the performance of...,전기 시장에서 정적 헤지와 동적 헤지의 성과를 추정하는 과정에서 이상한 결과를 얻었...,"첫 번째 질문에 대해 말씀드리자면, R-제곱 값이 상이하게 나타나는 이유는 회귀 분..."
414,Having to implement a replication strategy for...,유럽형 옵션의 복제 전략을 구현하는 과정에서 다음과 같은 문제가 발생하였습니다.\n...,"유럽형 옵션의 만기 시점 T에서는 옵션의 가치가 즉시 결정되며, 그에 따라 델타도 ..."
415,I am trying to bootstrap the 6m sterling swap ...,"6개월 스털링 스왑 곡선을 부트스트랩하기 위해, 주어진 예제에서 FRA(금리선도계약...",위 코드에서 FRA(금리선도계약)를 추가하기 위해서는 FRA 헬퍼를 생성할 수 있는...
416,I would like to price a n-th to default swap o...,n번째 디폴트 스왑을 5개의 자산 집합에 대해 가격을 책정하고자 합니다. R 코드로...,디폴트 스왑 가격 책정을 위해 시뮬레이션된 기본값 시리즈를 활용하는 방법에 대한 구...


In [42]:
df[df['response'] == 'Error: Rate limit exceeded or other issue']

,query,question,response
113,I was recently reading (and very much struggli...,"CBOE 스큐 지수에 대한 백서에서 설명된 내용을 바탕으로, 다음 질문에 답하십시오...",Error: Rate limit exceeded or other issue


In [30]:
df_no_duplicates = df[~df['response'].duplicated(keep=False)]

len(df_no_duplicates)

48

In [16]:
df[df.duplicated(subset='response')]

,query,question,response
3587,"Can the Hodges-Tompkins adjustment (S. Hodges,...","Hodges-Tompkins 조정(S. Hodges, R. Tompkins, The...",Error: Rate limit exceeded or other issue
3588,To price a call/put option with two possible f...,다음 질문에 답하십시오:\n\n1) 두 가지 가능한 미래 상태를 고려하여 콜/풋 옵...,Error: Rate limit exceeded or other issue
3589,I am trying to understand the abbreviations in...,유럽 주식 또는 STOXX 데이터에서 사용되는 약어에 대한 이해를 돕기 위해 다음 ...,Error: Rate limit exceeded or other issue
3590,Assume a model defined in a incomplete market....,불완전한 시장에서 정의된 모형을 가정하십시오. 이 모형은 매개변수 $\theta_1...,Error: Rate limit exceeded or other issue
3591,Examples like new research or findings but als...,"기계 학습이 금융 분야에 점차 확산되고 있는지에 대한 연구 현황을 고려할 때, 현재...",Error: Rate limit exceeded or other issue
...,...,...,...
3995,I need help in getting a more intuitive unders...,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue
3996,I am looking at the process \n$$X_t = \int_0^t...,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue
3997,If I made a GTC (good till cancelled) to a Sto...,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue
3998,I´m trying to build a Stock ranking/Picking mo...,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue


2. Qwen으로 질문 및 답변 생성 (너무 오래걸림)

In [1]:
import pandas as pd
import time
from tqdm import tqdm
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def qwen_completion(model, tokenizer, inputs, max_new_tokens=256):
    results = []
    for messages in inputs:
        # 메시지 포맷을 문자열로 변환
        prompt = ""
        for message in messages:
            if message["role"] == "system":
                prompt += f"[시스템]: {message['content']}\n"
            elif message["role"] == "user":
                prompt += f"[사용자]: {message['content']}\n"

        # Tokenize the prompt
        tokenized_inputs = tokenizer(prompt, return_tensors="pt")
        
        # 임베딩 레이어의 디바이스를 동적으로 가져옴
        embed_device = next(model.model.embed_tokens.parameters()).device
        
        # 입력 텐서를 임베딩 레이어의 디바이스로 이동
        tokenized_inputs = {key: val.to(embed_device) for key, val in tokenized_inputs.items()}

        with torch.no_grad():
            # Generate text using max_new_tokens
            outputs = model.generate(**tokenized_inputs, max_new_tokens=max_new_tokens)

        # Decode the generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        results.append(generated_text)

        # Explicitly free memory by deleting tensors and emptying the cache
        del tokenized_inputs, outputs
        torch.cuda.empty_cache()

    return results


# 메모리 사용량 확인
def print_memory_usage():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    print(f"Allocated Memory: {allocated:.2f} GB")
    print(f"Cached Memory: {reserved:.2f} GB")
    if allocated > 28:
        print("Warning: Allocated memory exceeds safe limit. Consider reducing batch size or max tokens.")

# 데이터셋 로드
ds = load_dataset('amphora/rewrite-se-quant')['train']
csv_file = 'qwen_data2.csv'

# 전체 데이터셋을 미리 데이터프레임으로 변환
df_ds = pd.DataFrame(ds)

# 데이터프레임 초기화
df = pd.DataFrame(columns=['query', 'question', 'response'])

# 모델과 토크나이저를 한 번만 로드
model_name = "Qwen/Qwen2-7B-Instruct"

# Mixed Precision(FP16) 사용 및 자동 오프로드 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 모델을 GPU에 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    offload_buffers=True,
    torch_dtype=torch.float16  # Mixed Precision 사용
).to(device)

print(torch.cuda.is_available())

# 배치 크기 설정
batch_size = 10  # 메모리 사용량 감소를 위해 배치 크기 조정

# 배치별로 질문과 답변 생성
for i in tqdm(range(0, len(df_ds), batch_size), desc="Processing Batches"):
    # 현재 배치의 데이터 선택
    batch_end = min(i + batch_size, len(df_ds))  # 마지막 인덱스를 넘어가지 않도록 조정

    # 질문 생성용 프롬프트 포맷팅
    batch_qrys = []
    for t in df_ds['query'][i:batch_end]:
        messages = [
            {"content": "Your job is creating quantitative finance questions in fluent Korean. You will be given a English QF question collected from the web. Restructure it to a test-like question, in formal Korean language. Return the question only.", "role": "system"},
            {"content": t, "role": "user"}
        ]
        batch_qrys.append(messages)

    # 생성된 질문
    question_resps = qwen_completion(
        model=model,
        tokenizer=tokenizer,
        inputs=batch_qrys,
        max_new_tokens=256
    )

    # 답변 생성용 프롬프트 구성
    batch_responses = []
    for question in question_resps:
        messages = [
            {"content": "Your job is to provide a formal, informative answer in Korean to the given finance-related question. The response should be accurate, concise, and written in a professional tone.", "role": "system"},
            {"content": f"질문: {question}", "role": "user"}
        ]
        batch_responses.append(messages)

    # 생성된 답변
    response_resps = qwen_completion(
        model=model,
        tokenizer=tokenizer,
        inputs=batch_responses,
        max_new_tokens=256
    )

    # 현재 배치의 데이터프레임 생성 및 기존 데이터프레임에 추가
    batch_df = pd.DataFrame({
        'query': df_ds['query'][i:batch_end].tolist(),
        'question': question_resps,
        'response': response_resps
    })

    df = pd.concat([df, batch_df], ignore_index=True)

    # 결과를 CSV 파일에 추가
    batch_df.to_csv(csv_file, mode='a', index=False, header=not bool(i), encoding='utf-8-sig')

    # 명시적으로 변수 삭제
    del batch_qrys, batch_responses, question_resps, response_resps
    torch.cuda.empty_cache()

# 최종 결과 확인
print(df.head())


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

True


Processing Batches:  44%|████▍     | 966/2195 [26:16:51<33:43:10, 98.77s/it] 

In [26]:
new_df.to_csv('output_dataset.csv', mode='w', index=False, encoding='utf-8-sig')

In [27]:
df = pd.read_csv('output_dataset.csv')

df

,query,question,response
0,"Since I am very new to this topic, i am strugg...",당신이 옵션 가격 책정과 현물 거래의 연관성을 이해하는 데 어려움을 겪고 있다고 하...,블랙-숄즈 모델을 활용하여 진입 및 청산 신호를 도출하는 방법은 여러 가지가 있을 ...
1,If I was to do a 99% VaR calculation on a port...,"포트폴리오의 수익률이 정규 분포 $\mathcal{N} (\mu, \sigma)$를...",1. **정의 및 99% VaR의 설명**\n VaR(가치-at-위험)은 주어진...
2,I'm writing my thesis on the relationship betw...,재무 성과와 지속 가능성 간의 관계에 관한 논문을 작성하고 있습니다. 이를 위해 두...,ESG 변수를 효과적으로 구축하기 위해 다음과 같은 방법을 고려해 보세요.\n\n1...
3,What sources of financial and economic data ar...,온라인에서 이용 가능한 금융 및 경제 데이터의 출처는 무엇인가요? 그 중 무료 또는...,"온라인에서 이용 가능한 금융 및 경제 데이터의 출처는 다양하며, 그 중 일부는 무료..."
4,Assume for now we are working in a stohastic v...,현재 우리는 확률적 변동성 모델(SV 설정)에서 작업하고 있다고 가정합시다. 다음과...,당신의 질문은 통계적 변동성 모델에서 옵션 가격의 민감도를 분석하는 데 중요한 포인...
...,...,...,...
8995,I'm working on a project where we're trying to...,"CMS(상수 만기 스왑)와 OIS(하룻밤 지수 스왑)가 서로 다른 개념인지, 혹은 ...",CMS(상수 만기 스왑)와 OIS(하룻밤 지수 스왑)는 본질적으로 서로 다른 금융 ...
8996,I see that the Fama-French library offers annu...,"Fama-French 모델의 연간 요인 데이터가 제공되는데도 불구하고, 왜 많은 사...",Fama-French 모델의 연간 요인 데이터가 제공되더라도 많은 연구자들이 월간 ...
8997,If there is deflation does the Inflation recei...,"디플레이션이 발생할 경우, 인플레이션 수신자는 고정금리 지급 외에도 감소된 소비자물...","디플레이션이 발생하면, 소비자물가지수(CPI)가 감소하게 되고, 이는 전반적인 가격..."
8998,I am aware that I can create a IRS in Quantlib...,주어진 함수 `makeSwap`을 이용하여 이자율 스왑(IRS)을 생성할 수 있음을...,기준 금리 스왑(basis swap)을 생성하기 위해서는 `makeSwap` 함수 ...


In [20]:
import pandas as pd

# 원하는 새로운 컬럼명 리스트
new_column_names = ['query', 'question', 'response']

# 변환 작업: 컬럼명을 데이터로 내리고, 새로운 데이터프레임 생성
data = {new_column_names[i]: [col] + df[col].tolist() for i, col in enumerate(df.columns)}

# 변환된 데이터프레임
new_df = pd.DataFrame(data)

new_df




,query,question,response
0,"Since I am very new to this topic, i am strugg...",당신이 옵션 가격 책정과 현물 거래의 연관성을 이해하는 데 어려움을 겪고 있다고 하...,블랙-숄즈 모델을 활용하여 진입 및 청산 신호를 도출하는 방법은 여러 가지가 있을 ...
1,If I was to do a 99% VaR calculation on a port...,"포트폴리오의 수익률이 정규 분포 $\mathcal{N} (\mu, \sigma)$를...",1. **정의 및 99% VaR의 설명**\n VaR(가치-at-위험)은 주어진...
2,I'm writing my thesis on the relationship betw...,재무 성과와 지속 가능성 간의 관계에 관한 논문을 작성하고 있습니다. 이를 위해 두...,ESG 변수를 효과적으로 구축하기 위해 다음과 같은 방법을 고려해 보세요.\n\n1...
3,What sources of financial and economic data ar...,온라인에서 이용 가능한 금융 및 경제 데이터의 출처는 무엇인가요? 그 중 무료 또는...,"온라인에서 이용 가능한 금융 및 경제 데이터의 출처는 다양하며, 그 중 일부는 무료..."
4,Assume for now we are working in a stohastic v...,현재 우리는 확률적 변동성 모델(SV 설정)에서 작업하고 있다고 가정합시다. 다음과...,당신의 질문은 통계적 변동성 모델에서 옵션 가격의 민감도를 분석하는 데 중요한 포인...
...,...,...,...
8995,I'm working on a project where we're trying to...,"CMS(상수 만기 스왑)와 OIS(하룻밤 지수 스왑)가 서로 다른 개념인지, 혹은 ...",CMS(상수 만기 스왑)와 OIS(하룻밤 지수 스왑)는 본질적으로 서로 다른 금융 ...
8996,I see that the Fama-French library offers annu...,"Fama-French 모델의 연간 요인 데이터가 제공되는데도 불구하고, 왜 많은 사...",Fama-French 모델의 연간 요인 데이터가 제공되더라도 많은 연구자들이 월간 ...
8997,If there is deflation does the Inflation recei...,"디플레이션이 발생할 경우, 인플레이션 수신자는 고정금리 지급 외에도 감소된 소비자물...","디플레이션이 발생하면, 소비자물가지수(CPI)가 감소하게 되고, 이는 전반적인 가격..."
8998,I am aware that I can create a IRS in Quantlib...,주어진 함수 `makeSwap`을 이용하여 이자율 스왑(IRS)을 생성할 수 있음을...,기준 금리 스왑(basis swap)을 생성하기 위해서는 `makeSwap` 함수 ...


In [21]:
print(new_df['question'][1])

포트폴리오의 수익률이 정규 분포 $\mathcal{N} (\mu, \sigma)$를 따르며, 변동성 $\sigma$가 감마 분포 $\Gamma(k, \theta)$를 따른다고 가정할 때, 99% VaR(가치-at-위험)을 계산하기 위한 방법을 상세히 설명하시오. 이를 위해 다음 단계를 따르세요:

1. 조건부 변동성 $\sigma$에 따라 정규 분포를 취급하여, 99% VaR의 정의를 설명하시오.
2. 변동성 $\sigma$가 담긴 포트폴리오 수익률의 분포에서 평균과 분산을 도출하시오.
3. Monte Carlo 시뮬레이션 기법을 활용하여, 감마 분포에 따라 샘플링한 변동성을 사용하여 VaR를 계산하는 방법을 설명하시오.
4. 최종적으로, 포트폴리오의 VaR를 추정하기 위한 코드를 작성하여, 프로그램을 통해 해당 VaR를 수치적으로 구하는 과정을 서술하시오.


In [22]:
print(new_df['response'][1])

1. **정의 및 99% VaR의 설명**
   VaR(가치-at-위험)은 주어진 신뢰 수준에서 포트폴리오의 손실이 예상되는 최대 금액을 나타내는 통계적 척도입니다. 99% VaR는 신뢰 수준이 99%인 상황에서 포트폴리오 손실이 최대 $L$ 이하일 확률이 99%라는 것을 의미합니다. 즉, 1%의 확률로 포트폴리오의 손실이 $L$을 초과할 것으로 예측됩니다. 만약 포트폴리오 수익률 $R$이 정규 분포 $\mathcal{N}(\mu, \sigma^2)$를 따르며, 변동성 $\sigma$가 감마 분포를 따른다고 가정할 경우, 우리는 VaR를 $R \sim \mathcal{N}(\mu, \sigma^2)$로 모델링할 수 있습니다.

2. **평균과 분산 도출**
   포트폴리오 수익률의 평균 및 분산은 조건부 변동성 $\sigma$에 따라 다음과 같이 표현됩니다:
   - 평균: $E[R | \sigma] = \mu$
   - 분산: $Var[R | \sigma] = \sigma^2$
   따라서 전체 분산을 구하기 위해서 변동성의 변동성을 고려해야 합니다. $\sigma$가 감마 분포 $\Gamma(k, \theta)$를 따른다면, $\sigma^2$는 차원감소(변동성의 제곱)를 적용하여 다음과 같이 얻을 수 있습니다.
   \[
   E[\sigma^2] = k \theta^2
   \]
   이 경우 포트폴리오 수익률 $R$의 평균과 분산은 다음과 같이 됩니다:
   - 평균: $E[R] = \mu$
   - 분산: $Var[R] = E[Var[R | \sigma]] + Var[E[R | \sigma]] = E[\sigma^2] = k \theta^2$

3. **Monte Carlo 시뮬레이션 기법**
   Monte Carlo 시뮬레이션을 통해 VaR를 계산하는 과정은 다음과 같습니다:
   1. 감마 분포 $\Gamma(k, \theta)$에서 변동성 $\sigma$를 $N$번 샘플링하여 $N$개의 변동성 샘플을 생성합니다.
   2. 각 샘플 $\sig

In [4]:
dff = pd.read_csv(csv_file)
dff

,query,question,response
0,I want to know how to price an American call o...,문제: I want to know how to price an American ca...,질문: 문제: I want to know how to price an America...
1,I'm attempting to calculate a GBP yield curve ...,문제: I'm attempting to calculate a GBP yield cu...,질문: 문제: I'm attempting to calculate a GBP yiel...
2,I want to calculate halflife with AR process a...,문제: I want to calculate halflife with AR proce...,질문: 문제: I want to calculate halflife with AR p...
3,Is call/put wing volatility smile calibration ...,문제: Is call/put wing volatility smile calibrat...,질문: 문제: Is call/put wing volatility smile cali...
4,I am trying understand and replicate this thes...,문제: I am trying understand and replicate this ...,질문: 문제: I am trying understand and replicate t...
...,...,...,...
4125,I am interested in an integrated call option t...,문제: I am interested in an integrated call opti...,질문: 문제: I am interested in an integrated call ...
4126,"For a project I need to fit a GARCH(1,1) model...","문제: For a project I need to fit a GARCH(1,1) m...","질문: 문제: For a project I need to fit a GARCH(1,..."
4127,I am investigating the Fama and French model u...,문제: I am investigating the Fama and French mod...,질문: 문제: I am investigating the Fama and French...
4128,Self-financing is an important concept in fina...,문제: Self-financing is an important concept in ...,질문: 문제: Self-financing is an important concept...


In [ ]:
print(f"중복된 행의 개수: {df.duplicated().sum()}")

In [ ]:
# 질문 생성 결과 길이 맞추기
if len(question_resps) < len(ds["query"]):
    question_resps.extend(["Error: Missing question"] * (len(ds["query"]) - len(question_resps)))

# 답변 생성 결과 길이 맞추기
if len(response_resps) < len(ds["query"]):
    response_resps.extend(["Error: Missing response"] * (len(ds["query"]) - len(response_resps)))


In [28]:
# Excel 파일 저장
# df.to_excel("output_path/result.xlsx")

# HuggingFace Hub 업로드 - token에 개인 HuggingFace 토큰을 입력해주시면 됩니다.
result_df = Dataset.from_pandas(new_df)
result_df.push_to_hub("llmflow/krx_dataset4", token=hftoken)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/llmflow/krx_dataset4/commit/d10376e4d22608f01971adad7c1f4cc1eda4cb38', commit_message='Upload dataset', commit_description='', oid='d10376e4d22608f01971adad7c1f4cc1eda4cb38', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/llmflow/krx_dataset4', endpoint='https://huggingface.co', repo_type='dataset', repo_id='llmflow/krx_dataset4'), pr_revision=None, pr_num=None)